## 사용자가 원하는 정보

1. 사용자의 질문과 관련있는 데이터

    - 관련이 있다는 것을 판단하는 기준은 vector가 된다.
    - vector : 단어 또는 문장의 유사도를 파악해서 관련성을 측정한다.

2. Vector를 생성하는 방법

    - Embedding 모델을 활용해서 vector 생성한다.
    - 문장에서 비슷한 단어가 자주 붙어있는 것을 학습한다.
        - 예를 들어, A : "왕은 왕자의 아버지다."  /  B : "여왕은 왕자의 어머니다."
        - "왕자의" 라는 단어 앞에 등장하는 "왕"과 "여왕"은 유사할 가능성이 높다.

## Vector Databse

1. Embedding 모델을 활용해 생성된 vector를 저장한다.

    - vector와 함꼐 metadata도 저장된다.
        - metadata : 문서의 이름, 페이지 번호 등의 데이터
    - hallucination을 대비하기 위해서 어떤 문서에서 가져왔는지 출처를 같이 보여주는 것이 중요하다.

2. vector를 대상으로 유사도 검색을 한다.

    - 사용자의 질문과 가장 비슷한 문서를 가져온다. (Retrieval)
        - 소득세법을 RAG의 knowledge base로 활용한다.
        - 문서 전체를 활용하게 되면 속도도 느리고, 토큰수 초과로 답변 생성이 안될 수 있다.
        - 문서를 chunking하는 작업이 매우 중요하다.
    
    - 가져온 문서를 prompt를 통해서 LLM에 제공한다. (Augmented)
    - LLM은 prompt를 활용해서 답변을 생성한다. (Generation)
    


### 예제

In [ ]:
# 1. 패키지 설치
#%pip install -q python-dotenv langchain-openai

In [ ]:
# 2. 환경변수 불러오기
#  .env 파일에 OPENAI_API_KEY 등록

import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# 3. oepnai client 불러오기

from openai import OpenAI
client = OpenAI()

In [ ]:
# 4. numpy를 사용해서 코사인 유사도 계산

import numpy as np
def cosine_similarity(vec1, vec2):
    """
    Compute the cosine similarity between two vectors.

    parameters:
    vec1 (numpy array) : First vector
    vec2 (numpy array) : Second vector

    Returns:
    float : Cosine similarity between vec1 and vec2
    """
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0
    
    return dot_product / (norm_vec1 * norm_vec2)

In [ ]:
# 5. "King" Embedding

king_embedding_response = client.embeddings.create(
    input="king",
    model="text-embedding-3-large"
)

king_vector = np.array(king_embedding_response.data[0].embedding)

In [ ]:
king_vector

array([ 0.01040417,  0.02499519, -0.0014776 , ...,  0.00835009,
        0.01049861, -0.00254005])

In [ ]:
# 6. "Queen" Embedding

queen_embedding_response = client.embeddings.create(
    input="queen",
    model="text-embedding-3-large"
)

queen_vector = np.array(queen_embedding_response.data[0].embedding)

In [ ]:
queen_vector

array([-0.01385735,  0.0008602 , -0.0167823 , ...,  0.00017693,
        0.01159847,  0.00638929])

In [ ]:
# 7. King vector와 Queen vector의 코사인 유사도 계산

similarity = cosine_similarity(king_vector, queen_vector)
print(similarity)

0.5552268369726672


In [ ]:
# 8. king과 slave의 유사도 계산

slave_embedding_response = client.embeddings.create(
    input="slave",
    model="text-embedding-3-large"
)
slave_vector = np.array(slave_embedding_response.data[0].embedding)
similarity_king_slave = cosine_similarity(king_vector, slave_vector)
print(similarity_king_slave)

0.2947745074537994


upstage api는 한국어에 성능이 더 좋다. 사용해보기!